In [1]:
import overpass
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')

import json
import csv

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

## City of Interest

In [2]:
#https://autogis-site.readthedocs.io/en/latest/lessons/lesson-6/retrieve-data-from-openstreetmap.html

min_longitude = -97.90787741442931 #note this is the bounding box for Austin, change to your desired location
min_latitude = 30.033924160573395

max_longitude = -97.55280269603111
max_latitude = 30.487698664291752

# query geometries
df_austin = ox.geometries_from_bbox(
    min_latitude, max_latitude, min_longitude, max_longitude,
    tags={"amenity": ["library", "community center", "senior", "recreation center"]}
)

print(f"Number of cooling centers found: {len(df_austin)}")

Number of cooling centers found: 36


In [3]:
# only keep these topics
interesting_columns = [
    'geometry',
    "name",
    "addr:housenumber",
    "addr:street",
    "addr:city",
    "addr:postcode",
]
original_crs = df_austin.crs

target_crs = 'EPSG:6587' #use unique crs to your region
df_austin = df_austin.to_crs(target_crs)

#some of the places it read in were polygons, not points
#just took the centroid of those shapes
# centroids
polygons_df_austin = df_austin[df_austin['geometry'].geom_type == 'Polygon']
centroids = polygons_df_austin['geometry'].centroid
df_austin.loc[polygons_df_austin.index, 'geometry'] = centroids

#revert back to original crs
df_austin = df_austin.to_crs(original_crs)

df_austin.head()

addr:state  amenity  ele gnis:feature_id  \
element_type osmid                                                 
node         368162040          TX  library  163         2360810   
             2667102948        NaN  library  NaN             NaN   
             4826271539        NaN  library  NaN             NaN   
             8093097715        NaN  library  NaN             NaN   
             9956137736         TX  library  194         2360790   

                                                            name  \
element_type osmid                                                 
node         368162040                   Texas State Law Library   
             2667102948                         Garfield Library   
             4826271539                Lantana Free Tiny Library   
             8093097715              Westlake Highschool Library   
             9956137736  Austin Public Library - St. John Branch   

                                source                    geometry   access  \
element_type osmid                                                            
node         368162040   USGS Geonames  POINT (-97.74198 30.27624)      NaN   
             2667102948            NaN  POINT (-97.56057 30.18788)      NaN   
             4826271539            NaN  POINT (-97.87417 30.24811)      NaN   
             8093097715            NaN  POINT (-97.81406 30.27601)  private   
             9956137736  USGS Geonames  POINT (-97.69372 30.33205)      NaN   

                        air_conditioning baby_feeding  ... toilets:wheelchair  \
element_type osmid                                     ...                      
node         368162040               NaN          NaN  ...                NaN   
             2667102948              NaN          NaN  ...                NaN   
             4826271539              NaN          NaN  ...                NaN   
             8093097715              yes           no  ...                NaN   
             9956137736              NaN          NaN  ...                NaN   

                        name:en tourism wikimedia_commons layer  \
element_type osmid                                                
node         368162040      NaN     NaN               NaN   NaN   
             2667102948     NaN     NaN               NaN   NaN   
             4826271539     NaN     NaN               NaN   NaN   
             8093097715     NaN     NaN               NaN   NaN   
             9956137736     NaN     NaN               NaN   NaN   

                        opening_hours:covid19 operator:wikidata branch height  \
element_type osmid                                                              
node         368162040                    NaN               NaN    NaN    NaN   
             2667102948                   NaN               NaN    NaN    NaN   
             4826271539                   NaN               NaN    NaN    NaN   
             8093097715                   NaN               NaN    NaN    NaN   
             9956137736                   NaN               NaN    NaN    NaN   

                        coa:place_id  
element_type osmid                    
node         368162040           NaN  
             2667102948          NaN  
             4826271539          NaN  
             8093097715          NaN  
             9956137736          NaN  

[5 rows x 44 columns]

In [4]:
#had to turn it into csv to seperate the geometry column into lat long coordinates

df_austin.to_csv('temporary_Austin.csv', index=False)
df_austin = pd.read_csv('temporary_Austin.csv')


In [5]:
#keep this info
df_austin = df_austin.loc[:, ['geometry',
    "name",
    "addr:housenumber",
    "addr:street",
    "addr:city",
    "addr:postcode"]]

df_austin

,geometry,name,addr:housenumber,addr:street,addr:city,addr:postcode
0,POINT (-97.741983 30.276236400000005),Texas State Law Library,NaN,NaN,NaN,NaN
1,POINT (-97.5605691 30.18788339999999),Garfield Library,NaN,NaN,NaN,NaN
2,POINT (-97.87416700000001 30.248106099999966),Lantana Free Tiny Library,NaN,NaN,NaN,NaN
3,POINT (-97.8140618 30.276009800000008),Westlake Highschool Library,NaN,NaN,NaN,NaN
4,POINT (-97.6937228 30.332052000000004),Austin Public Library - St. John Branch,7500.0,Blessing Avenue,Austin,78752.0
5,POINT (-97.83716060000002 30.086859600000007),Buda Public Library,405.0,East Loop Street,Buda,78610.0
6,POINT (-97.73821371083662 30.282735107376194),Perry-Castañeda Library,101.0,East 21st Street,NaN,78705.0
7,POINT (-97.73304503019432 30.27876799152096),Collections Deposit Library,NaN,NaN,NaN,NaN
8,POINT (-97.74072810865192 30.32346091625376),Austin Public Library - Yarborough Branch,2200.0,Hancock Drive,Austin,78756.0
9,POINT (-97.73842416892374 30.27389765269059),Lorenzo de Zavala State Archive and Library,NaN,NaN,NaN,NaN


In [6]:
#convert it to string so that we can extract just the #s out of POINT (# #)
df_austin['geometry'].astype(str)

def extract_coordinates(point_str):
    point_str = point_str.replace('POINT (', '').replace(')', '') 
    coordinates = point_str.split() 
    return float(coordinates[0]), float(coordinates[1])

# make long, lat columns
df_austin[['Y', 'X']] = pd.DataFrame(df_austin['geometry'].apply(extract_coordinates).tolist())

df_austin = df_austin.drop(columns = ['geometry','addr:housenumber', 'addr:street', 'addr:city', 'addr:postcode'])

In [7]:
df_austin.to_csv('austin_cooling_OSM.csv', index=False)